# 01_EDA - Exploratory Data Analysis
Este cuaderno contiene análisis exploratorio de los datos (ERA5, CHIRPS, MODIS).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Cargar datos con src.utils.data_loader.load_data
# datos = load_data('..\data\raw')